# How-to Guide: Correlations

Dans le fichier `cc_tk.feature.correlation`, 3 méthodes de sélection de variables à partir des corrélations sont implémentées. Ces 3 méthodes sont implémentées au travers de l'interface `TransformerMixin` de `sklearn` pour pouvoir être intégrée facilement dans des pipelines.

1. Corrélation avec la variable cible (`CorrelationToTarget`) : on garde uniquement les variables qui sont corrélées avec la variable à prédire. Le seuil de corrélation est un paramètre de ce transformer.
2. Détection des corrélations par paires (`PairwiseCorrelationDrop`) : quand une variable est corrélée (seuil de corrélation en paramètre), on retire cette variable de notre sélection. Par symétrie, on devrait retirer les deux variables en questions, le choix qui est fait est de retirer celle qui est en moyenne la plus corrélée avec les autres variables. Cette méthode est expliquée [ici](https://towardsdatascience.com/are-you-dropping-too-many-correlated-features-d1c96654abe6) et étendue pour éviter de retirer trop de colonnes. L'extension repose sur le fait que l'on retire itérativement les variables corrélées pour éviter de les prendre en compte lorsque l'on s'intéresse aux variables suivantes.
3. Regroupement de variables en fonction de leurs corrélations (`ClusteringCorrelation`) : les variables sont regroupés par une CAH en prenant comme distance $1-abs(corr(X_i, X_j))$. Les groupes sont déterminés par un seuil de corrélation, toutes les variables dans un groupe sont corrélées à plus de $1 - seuil$. La méthode est détaillée [ici](https://kobia.fr/automatiser-la-reduction-des-correlations-par-clustering/). Pour chaque groupe, 2 possibilités : on garde un nombre de colonnes fixé à l'avance (1 par défaut), on construit une PCA pour chaque groupe et on choisit le nombre de composantes à garder.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import set_config

from sklearn.datasets import load_diabetes
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

from cc_tk.feature.correlation import (
    CorrelationToTarget,
    ClusteringCorrelation,
    PairwiseCorrelationDrop,
)

In [ ]:
set_config(transform_output="pandas")

In [ ]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [ ]:
X.columns

In [ ]:
baseline = DummyRegressor()
baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

## Corrélation avec la cible

In [ ]:
correlation_target_pipeline = make_pipeline(
    MinMaxScaler().set_output(transform="pandas"),
    CorrelationToTarget(threshold=0.1),
    LinearRegression(),
)
correlation_target_pipeline.fit(X_train, y_train)
correlation_target_pipeline.score(X_test, y_test)

In [ ]:
correlation_target_pipeline["correlationtotarget"]._selected_columns

In [ ]:
correlation_target_pipeline["correlationtotarget"].plot_correlation()

## Corrélation par pair

In [ ]:
correlation_pairwise_pipeline = make_pipeline(
    MinMaxScaler().set_output(transform="pandas"),
    PairwiseCorrelationDrop(threshold=0.8),
    LinearRegression(),
)
correlation_pairwise_pipeline.fit(X_train, y_train)
correlation_pairwise_pipeline.score(X_test, y_test)

In [ ]:
correlation_pairwise_pipeline["pairwisecorrelationdrop"]._columns_selection

## Clusters de corrélation

Il y a deux `summary_method` disponibles:
- la valeur par défaut `first` qui utilise uniquement les premières variables de chaque cluster. On peut voir les variables retenues avec l'attribut `_selected_columns_`
- `pca` qui utilise l'analyse en composantes principales pour résumer les clusters. On peut voir les colonnes en output avec l'attribut `_output_columns`

In [ ]:
correlation_clustering_pipeline = make_pipeline(
    MinMaxScaler().set_output(transform="pandas"),
    ClusteringCorrelation(threshold=0.8, summary_method="pca", n_variables_by_cluster=2),
    LinearRegression(),
)
correlation_clustering_pipeline.fit(X_train, y_train)
correlation_clustering_pipeline.score(X_test, y_test)

In [ ]:
dendro = correlation_clustering_pipeline["clusteringcorrelation"].plot_dendro()

In [ ]:
correlation_clustering_pipeline["clusteringcorrelation"]._output_columns
# correlation_clustering_pipeline["clusteringcorrelation"]._selected_columns_   # When summary_method="first"

In [ ]:
correlation_clustering_pipeline[
    "clusteringcorrelation"
].plot_correlation_matrix()

## Combinaison de différents transformer

Il est aussi possible de combiner différentes méthodes de sélection de variables.

In [ ]:
correlation_combined_pipeline = make_pipeline(
    MinMaxScaler().set_output(transform="pandas"),
    CorrelationToTarget(threshold=0.1),
    ClusteringCorrelation(threshold=0.3, summary_method="first"),
    LinearRegression(),
)
correlation_combined_pipeline.fit(X_train, y_train)
correlation_combined_pipeline.score(X_test, y_test)